In [1]:
import tensorflow as tf
from tensorflow import keras
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import time
import string
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from io import BytesIO

2022-12-07 14:50:57.509438: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 14:50:57.509453: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from gensim.models import Word2Vec

In [3]:
from nltk.corpus import stopwords 
import string
from nltk import word_tokenize 
import nltk
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import gensim.downloader as api
from keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/lscr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import pandas as pd
import numpy as np

In [5]:
new_model = tf.keras.models.load_model('../models/model2')

2022-12-07 14:51:00.067126: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-07 14:51:00.067146: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-07 14:51:00.067160: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lscr): /proc/driver/nvidia/version does not exist
2022-12-07 14:51:00.067338: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 14:51:02.029878: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 1

In [6]:
def clean1 (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    lowercased=lowercased.replace('\n','')
    return lowercased

In [7]:
def get_photo_text(url):
    
    # initialisation of the webdriver 
    options = Options()
    options.headless = True
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get(url)
    
    # taking the screenshot
    screenshot = driver.get_screenshot_as_png()
    screenshot = Image.open(BytesIO(screenshot))
    screenshot = np.array(screenshot)[:, :, :3]
    screenshot = screenshot/255.
    screenshot = tf.image.resize(screenshot, (300, 400))

    # scrape the text
    text = driver.find_elements( by = "tag name", value = "body")
    final1 = text[0].get_attribute("innerText")
    final_text=clean1(final1)

    return screenshot, final_text

In [21]:
urls = [
    'https://www.fxnewera.com/',
    'https://blufx.co.uk/',
    'https://go2loan.online/',
    'http://www.coinvaluebankonline.com/',
    'https://www.onlinecnpbank.com/',
    
    'https://www.wahed.com/',
    'https://www.westerncircle.co.uk/',
    'https://www.zodia.io/',
    'https://bytetreeam.com/',
    'http://www.ratesnet.com/'
]

X_images = []
X_texts = []
for url in urls:
    image_, text_ = get_photo_text(url)
    
    X_images.append(image_)
    X_texts.append(text_)
    
X_images = np.array(X_images)

/tmp/ipykernel_46581/2084395275.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [22]:
X_images.shape

(10, 300, 400, 3)

In [23]:
# text, image = get_photo_text(url)

In [24]:
def clean(text):
    text = text.split()
    words_only = [word for word in text if word.isalpha()]
    for punctuation in string.punctuation:
        words_only = [word.replace(punctuation, ' ').lower() for word in words_only] # Remove Punctuation
    # lowercased = text.lower() # Lower Case
    #tokenized = word_tokenize(lowercased) # Tokenize
    # words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    # lemma=WordNetLemmatizer() # Initiate Lemmatizer
    # lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return without_stopwords

def text_cleaner(list_text):
    list_clean_text=[]
    for text in list_text:
        #print(text)
        cleen_txt=clean(text)#[0]
        list_clean_text.append(cleen_txt)
        #print(list_clean_text)
    return list_clean_text

def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed


def preprocess_pad(list_text):
    list_ = text_cleaner(list_text)
    
    word2vec_transfer = api.load('glove-wiki-gigaword-100')
    
    list_ = embedding(word2vec_transfer, list_)
    list_ = pad_sequences(list_, dtype='float32', padding='post', maxlen=200)
    return list_
        

In [25]:
X_texts_pad = preprocess_pad(X_texts)

In [26]:
X_texts_pad.shape

(10, 200, 100)

In [27]:
prediction = new_model.predict([X_images, X_texts_pad])

In [30]:
prediction[0][0]

0.591569